You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: The Future of Artificial Intelligence: Trends, Players, and News

Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in today's world

II. Latest Trends in Artificial Intelligence
- Machine learning advancements
- Natural language processing developments
- Automation and robotics in AI

III. Key Players in the AI Industry
- Google
- Amazon
- Microsoft
- IBM

IV. Noteworthy News in Artificial Intelligence
- Breakthroughs in AI healthcare applications
- E

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) is a rapidly evolving field that involves the development of intelligent machines capable of performing tasks that typically require human intelligence. From self-driving cars to virtual assistants, AI has become an integral part of our daily lives. The importance of AI in today's world cannot be understated, as it has the potential to revolutionize industries, improve efficiency, and drive innovation.

## Latest Trends in Artificial Intelligence
One of the latest trends in AI is the advancements in machine learning, where algorithms are constantly improving their performance without being explicitly programmed. This has led to significant breakthroughs in various applications, such as image recognition, natural language processing, and recommendation systems. Additionally, developments in natural language processing have enabled 

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# The Benefits of Incorporating Meditation into Daily Routine

In today's fast-paced and stressful world, the practice of meditation has become increasingly popular as a means of promoting mental health and overall well-being. With key players in the industry like Headspace and Calm leading the way, more individuals are turning to mindfulness practices to combat the negative effects of stress and anxiety. Studies have shown the positive impact of meditation on mental health, making it a valuable tool for young professionals struggling with high stress levels and a lack of work-life balance.

## Stress Reduction
One of the most significant benefits of incorporating meditation into a daily routine is its ability to reduce stress levels. Research studies have consistently shown the effectiveness of meditation in calming the mind and promoting relaxation. Many individuals have shared personal anecdotes of how meditation has helped them cope with the pressures of daily life, allowing them to find a sense of peace and calm amidst chaos.

## Improved Focus and Concentration
In addition to stress reduction, meditation has been found to improve focus and concentration. Scientific evidence suggests that regular meditation practice can enhance cognitive function, making it easier to stay present and attentive in various tasks. By incorporating short meditation practices into daily routines, individuals can experience increased clarity of mind and improved productivity throughout the day.

## Enhanced Emotional Well-being
Another key benefit of meditation is its ability to enhance emotional well-being. By practicing mindfulness and self-awareness, individuals can learn to manage their emotions more effectively and promote overall mental health. Testimonials from those who have integrated meditation into their lives highlight the positive impact it has had on their emotional resilience and ability to navigate challenging situations with grace.

In conclusion, the benefits of incorporating meditation into daily routine are vast and impactful. From stress reduction to improved focus and emotional well-being, mindfulness practices offer a holistic approach to mental health and personal development. As we navigate the demands of modern life, establishing a daily meditation practice can be a transformative tool for finding balance and inner peace. I encourage you to explore guided meditation sessions or local classes to begin your journey towards a healthier and more mindful lifestyle.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [14]:
topic = "Whaam"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Whaam.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: Exploring the Impact of Whaam in the Art World

Outline:
I. Introduction
- Brief overview of the artwork "Whaam" by Roy Lichtenstein
- Importance of understanding the context and significance of the piece
- Preview of key points to be discussed

II. Latest Trends and Key Players
- Analysis of recent exhibitions featuring "Whaam"
- Overview of art critics' perspectives on the impact of the artwork
- In-depth look at the influence of Roy Lichtenstein on the Pop Art movement

III. Noteworthy News on Whaam


I now can give a great answer

Final Answer:

# Exploring the Impact of Whaam in the Art World

## Introduction

The iconic artwork "Whaam" by Roy Lichtenstein has left a lasting impression on the art world since its creation in the early 1960s. This piece, inspired by comic book imagery, is a prime example of Lichtenstein's unique Pop Art style. Understanding the context and significance of "Whaam" is crucial to fully appreciate its impact on the art world. In this blog post, we will delve into the latest trends and key players in the art scene, explore noteworthy news surrounding "Whaam," analyze its influence on the Pop Art movement, and identify the target audience interested in this renowned artwork.

## Latest Trends and Key Players

Recent exhibitions featuring "Whaam" have continued to attract art enthusiasts and critics alike. The vibrant colors and bold lines of the artwork have captivated audiences around the world, solidifying its place as a timeless masterpiece. Art critic

In [15]:
Markdown(result)

# Exploring the Impact of Whaam in the Art World

## Introduction

The iconic artwork "Whaam" by Roy Lichtenstein has left a lasting impression on the art world since its creation in the early 1960s. This piece, inspired by comic book imagery, is a prime example of Lichtenstein's unique Pop Art style. Understanding the context and significance of "Whaam" is crucial to fully appreciate its impact on the art world. In this blog post, we will delve into the latest trends and key players in the art scene, explore noteworthy news surrounding "Whaam," analyze its influence on the Pop Art movement, and identify the target audience interested in this renowned artwork.

## Latest Trends and Key Players

Recent exhibitions featuring "Whaam" have continued to attract art enthusiasts and critics alike. The vibrant colors and bold lines of the artwork have captivated audiences around the world, solidifying its place as a timeless masterpiece. Art critics have praised "Whaam" for its innovative approach to blending popular culture with fine art, highlighting Lichtenstein's role as a trailblazer in the Pop Art movement. By examining the influence of Roy Lichtenstein on his contemporaries and the art world at large, we can gain a deeper appreciation for the lasting impact of "Whaam."

## Noteworthy News on Whaam

Updates on recent auctions or sales of "Whaam" shed light on the artwork's continued relevance in the contemporary art market. Despite controversies or debates surrounding the commercialization of Pop Art, "Whaam" remains a symbol of artistic innovation and cultural significance. By exploring the ongoing discussions and interpretations of this iconic piece, we can better understand its enduring legacy in the art world. The relevance of "Whaam" in contemporary art serves as a testament to Lichtenstein's ability to transcend time and connect with audiences across generations.

## Target Audience Analysis

Art enthusiasts, students, collectors, and curious individuals make up the diverse target audience interested in "Whaam" and the works of Roy Lichtenstein. Whether studying art history, seeking investment opportunities, or simply exploring renowned artworks, readers will find valuable insights in this blog post. By engaging with the themes and techniques present in "Whaam," audiences can deepen their understanding of Pop Art and modern art movements, fostering a greater appreciation for Lichtenstein's contributions to the art world.

In conclusion, "Whaam" by Roy Lichtenstein continues to hold a prominent place in the art world, captivating audiences with its bold imagery and cultural commentary. By exploring the impact of this iconic artwork, we can gain a deeper appreciation for Lichtenstein's legacy and the enduring relevance of Pop Art in contemporary society. I encourage readers to visit galleries and museums to view "Whaam" in person, engage in discussions on social media, and explore more artworks by Lichtenstein and other Pop Art artists to further enrich their artistic experience.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).